#  Tanzania Water Pump Failure Predictions - Competition

## <font color=green>Implement a Predictive Model to help solve this problem and submit to DRIVENDATA competition</font>



>  **This is a WORK-IN-PROGRESS -   10/26/2022 - please check back for the final version...**

...


Can you predict which water pumps are faulty?

Using data from Taarifa and the Tanzanian Ministry of Water, 
can you predict which pumps are functional, which need some repairs, 
and which don't work at all? Predict one of these three classes
based on a number of variables about what kind of pump is operating, 
when it was installed, and how it is managed. 

A smart understanding of which waterpoints will fail can improve maintenance operations 
and ensure that clean, potable water is available to communities across Tanzania.

...

Hope this simple example of an XGBoost predictive model helps!

All the best,
Mike Pastor


**Competition Home Page:**  
[DRIVENDATA - Data science competitions
to build a better world](https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/page/23/)



#  First, let's load all of the necessary Python libraries
#    ( Note: these need to be installed on the Python instance running the NoteBook)


In [6]:


import pandas as pd # load and manipulate data and for One-Hot Encoding
import numpy as np # calculate the mean and standard deviation
import xgboost as xgb # XGBoost stuff
from sklearn.model_selection import train_test_split # split  data into training and testing sets
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer # for scoring during cross validation
from sklearn.model_selection import GridSearchCV # cross validation
from sklearn.metrics import confusion_matrix # creates a confusion matrix
from sklearn.metrics import plot_confusion_matrix # draws a confusion matrix


#  Now, let's open the Datasets 


In [7]:

# load TRAIN datasets ###########################################################
#
train_X = pd.read_csv('./TrainingSetValues.csv')
train_Y = pd.read_csv('./TrainingSetLABELS.csv')

# Merge the Y onto the X to be sure they are matched
#  This adds 'status_group' LABEL onto each X example row
#
train_X = train_X.merge( train_Y, how='inner', on='id' )



#########################################################
# replacing Y  string values with numerics
#    This is becomes part of our Tanzania Data Dictionary
#
train_X['status_group'].replace(\
    ['functional', 'functional needs repair', 'non functional'],\
    [0, 1, 2], inplace=True)

# Used the synchronized - quantitative  LABEL for Y
train_Y = pd.DataFrame( train_X['status_group'], columns=['status_group'] )


########################################################
#  Also  Load the TEST dataset which is used for the Submission preparation
#
test_X = pd.read_csv('./TestSetValues.csv')


#  Take a subset for testing
train_X = train_X[ [    'amount_tsh', 'gps_height', 'longitude', 'latitude', 'population', 'construction_year' ] ]
#                         # 'latitude', 'longitude', 'basin', 'region_code', 'district_code', 'population', \
#                         # 'scheme_name', 'scheme_management', 'construction_year', 'quantity',   'payment', \
#                         # 'extraction_type', 'extraction_type_class', 'source', 'source_type', \
#                         'management', 'waterpoint_type'  ]  ]


# Save IDs for Submission file preparation below
test_X_Saved_IDs = test_X[ [  'id' ] ]

test_X = test_X[ [      'amount_tsh', 'gps_height', 'longitude', 'latitude', 'population', 'construction_year' ] ]
                        # 'latitude', 'longitude', 'basin', 'region_code', 'district_code', 'population', \
                        # 'scheme_name', 'scheme_management', 'construction_year', 'quantity',   'payment', \
                        # 'extraction_type', 'extraction_type_class', 'source', 'source_type', \
                        # 'management', 'waterpoint_type'  ]  ]


print( train_X.dtypes )


print('Datasets Loaded & Merged...   Y - X ', train_Y.shape, train_X.shape )


amount_tsh           float64
gps_height             int64
longitude            float64
latitude             float64
population             int64
construction_year      int64
dtype: object
Datasets Loaded & Merged...   Y - X  (59400, 1) (59400, 6)
